In [151]:
#import and setup
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
from pprint import pprint
from datetime import datetime

finn_key = 'sandbox_cc7v412ad3i03fardc9g'

#import finnhub
import finnhub
finnhub_client = finnhub.Client(api_key=finn_key)

#finnhub utcdate codes for slicing by year for each stock
#2019 times
beg_2019 = 1546300800
end_2019 = 1577836799
#2020 times
beg_2020 = 1577836800
end_2020 = 1609459199
#2021 times
beg_2021 = 1609459200
end_2021 = 1640995199


In [174]:
#def function to create a dataframe for each stock
def stock_df(stock_name, beg_year, end_year):
    df_name = pd.DataFrame(finnhub_client.stock_candles(stock_name, 'D', beg_year, end_year)) 
    df_name = df_name.drop(['s'], axis=1)
    rename = {'c':'Close','h': 'High', 'l': 'Low', 'o': 'Open', 't': 'Time', 'v': 'Volume'}
    df_name.rename(columns=rename, inplace=True)
    df_name['Time'] = [datetime.utcfromtimestamp(x).strftime('%Y-%m-%d') for x in df_name['Time']]
    return df_name

In [ ]:
def plot_chart(dataframe):
    return('hello no plotting done yet')

In [175]:
test = stock_df('MSFT',beg_2020, end_2020)
test

,Close,High,Low,Open,Time,Volume
0,123.282900,123.492000,122.631000,123.000000,2020-01-01,1
1,123.837673,124.047714,123.182840,123.553500,2020-01-02,75
2,124.394943,124.605929,123.737162,124.109491,2020-01-03,99
3,124.480981,125.166655,124.293980,124.667983,2020-01-04,45
4,124.232020,124.916322,124.045392,124.418647,2020-01-05,67
...,...,...,...,...,...,...
361,177.434371,178.411726,177.167820,177.700922,2020-12-27,96
362,177.079502,178.054903,176.813484,177.345521,2020-12-28,51
363,176.725343,177.698793,176.459857,176.990829,2020-12-29,98
364,177.043113,177.343395,176.106937,176.636848,2020-12-30,74


In [153]:
microsoft = stock_df('MSFT', beg_2020, end_2020)
microsoft

,c,h,l,o,s,t,v
0,123.282900,123.492000,122.631000,123.000000,ok,1577836800,83
1,123.368170,124.047714,123.182840,123.553500,ok,1577923200,57
2,123.589998,123.799619,122.936474,123.306393,ok,1578009600,44
3,124.146153,124.356717,123.489688,123.861272,ok,1578096000,58
4,124.704810,124.916322,124.045392,124.418647,ok,1578182400,73
...,...,...,...,...,...,...,...
361,215.005716,215.370387,213.868801,214.512338,ok,1609027200,98
362,215.973242,216.339554,214.831211,215.477643,ok,1609113600,89
363,216.945122,217.313082,215.797951,216.447293,ok,1609200000,18
364,217.095174,218.290991,216.769042,217.421306,ok,1609286400,72


In [158]:

# for index, row in micro_clean.iterrows():
#     convert_most_recent = datetime.utcfromtimestamp(row['Time']).strftime('%Y-%m-%d')
#     # row['Time'] = convert_most_recent
#     micro_clean['Time'] = str(row['Time']).replace(str(row['Time']), convert_most_recent)
#     print(row['Time'])